## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Import the csv 
import os
os.chdir('..')

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Paris,48.8534,2.3488,45.00,100,90,9.22,FR,2021-02-07 01:56:18,drizzle
1,1,Bismil,37.8486,40.6658,35.60,100,0,3.65,TR,2021-02-07 01:57:10,clear sky
2,2,Qaanaaq,77.4840,-69.3632,10.69,87,100,7.56,GL,2021-02-07 01:57:10,overcast clouds
3,3,Katobu,-4.9408,122.5276,84.87,72,100,4.74,ID,2021-02-07 01:54:00,overcast clouds
4,4,Mar Del Plata,-38.0023,-57.5575,64.00,77,20,11.50,AR,2021-02-07 01:55:13,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
3,3,Katobu,-4.9408,122.5276,84.87,72,100,4.74,ID,2021-02-07 01:54:00,overcast clouds
8,8,Rikitea,-23.1203,-134.9692,78.78,69,73,10.56,PF,2021-02-07 01:55:25,broken clouds
9,9,Hilo,19.7297,-155.0900,78.80,53,75,5.75,US,2021-02-07 01:53:24,broken clouds
13,13,Alofi,-19.0595,-169.9187,86.00,70,100,12.66,NU,2021-02-07 01:54:13,light rain
14,14,Cayenne,4.9333,-52.3333,77.00,100,90,8.05,GF,2021-02-07 01:57:12,mist
16,16,Takoradi,4.8845,-1.7554,79.57,86,0,11.90,GH,2021-02-07 01:57:12,clear sky
19,19,Ca Mau,9.1769,105.1500,82.58,62,87,7.76,VN,2021-02-07 01:57:13,overcast clouds
22,22,Butaritari,3.0707,172.7902,80.98,81,71,20.78,KI,2021-02-07 01:54:01,broken clouds
24,24,Mataura,-46.1927,168.8643,81.88,46,13,1.63,NZ,2021-02-07 01:52:29,few clouds
38,38,Belmonte,-15.8631,-38.8828,78.80,83,75,4.61,BR,2021-02-07 01:57:15,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()
#preferred_cities_df.count()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Current Description", "Lng"]].dropna().copy()
clean_df["Hotel Name"] = ""
clean_df.head()

,City,Country,Max Temp,Lat,Current Description,Lng,Hotel Name
3,Katobu,ID,84.87,-4.9408,overcast clouds,122.5276,
8,Rikitea,PF,78.78,-23.1203,broken clouds,-134.9692,
9,Hilo,US,78.80,19.7297,broken clouds,-155.0900,
13,Alofi,NU,86.00,-19.0595,light rain,-169.9187,
14,Cayenne,GF,77.00,4.9333,mist,-52.3333,


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Katobu,ID,84.87,overcast clouds,-4.9408,122.5276,
8,Rikitea,PF,78.78,broken clouds,-23.1203,-134.9692,
9,Hilo,US,78.80,broken clouds,19.7297,-155.0900,
13,Alofi,NU,86.00,light rain,-19.0595,-169.9187,
14,Cayenne,GF,77.00,mist,4.9333,-52.3333,
16,Takoradi,GH,79.57,clear sky,4.8845,-1.7554,
19,Ca Mau,VN,82.58,overcast clouds,9.1769,105.1500,
22,Butaritari,KI,80.98,broken clouds,3.0707,172.7902,
24,Mataura,NZ,81.88,few clouds,-46.1927,168.8643,
38,Belmonte,BR,78.80,broken clouds,-15.8631,-38.8828,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Katobu,ID,84.87,overcast clouds,-4.9408,122.5276,La Ode Husein
8,Rikitea,PF,78.78,broken clouds,-23.1203,-134.9692,Pension Maro'i
9,Hilo,US,78.80,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Alofi,NU,86.00,light rain,-19.0595,-169.9187,Taloa Heights
14,Cayenne,GF,77.00,mist,4.9333,-52.3333,Hôtel Le Dronmi
16,Takoradi,GH,79.57,clear sky,4.8845,-1.7554,Raybow International Hotel
19,Ca Mau,VN,82.58,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
22,Butaritari,KI,80.98,broken clouds,3.0707,172.7902,Isles Sunset Lodge
24,Mataura,NZ,81.88,few clouds,-46.1927,168.8643,Ellie's Villa
38,Belmonte,BR,78.80,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão


In [10]:
# Determine if there is any rows.
hotel_df.isna().sum()


City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Katobu,ID,84.87,overcast clouds,-4.9408,122.5276,La Ode Husein
8,Rikitea,PF,78.78,broken clouds,-23.1203,-134.9692,Pension Maro'i
9,Hilo,US,78.80,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Alofi,NU,86.00,light rain,-19.0595,-169.9187,Taloa Heights
14,Cayenne,GF,77.00,mist,4.9333,-52.3333,Hôtel Le Dronmi
16,Takoradi,GH,79.57,clear sky,4.8845,-1.7554,Raybow International Hotel
19,Ca Mau,VN,82.58,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
22,Butaritari,KI,80.98,broken clouds,3.0707,172.7902,Isles Sunset Lodge
24,Mataura,NZ,81.88,few clouds,-46.1927,168.8643,Ellie's Villa
38,Belmonte,BR,78.80,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))